In [12]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import CRPS.CRPS as pscore


pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from PCR_evaluation.dumas_evaluation import *

In [13]:
with open('../../../data/PCR/transformed_event_logs/PCR_test.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

with open('../../../data/PCR/transformed_event_logs/PCR_train.pickle', 'rb') as f:
    train_event_log = pickle.load(f)

In [14]:
import sys
sys.path.append('../../../src/TaskExecutionTimeMining/pix-framework-main/src')
with open('./dumas_model_no_resource.pickle', 'rb') as f:
    dumas_model = pickle.load(f)

In [15]:
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

N = 1000
n_processes = 50

In [16]:
import conduct_evaluation

evaluator = conduct_evaluation.ConductEvaluation(dumas_model, SampleOutcomes_Dumas_Normal, 
                                                 {},
                                     test_event_log, n=N)

likelihoods_test = evaluator.sample_cases(plot_cases=False, multiprocessing=True)

100%|██████████| 4960/4960 [01:19<00:00, 62.01it/s] 


In [19]:
import conduct_evaluation

evaluator = conduct_evaluation.ConductEvaluation(dumas_model, SampleOutcomes_Dumas_Normal, 
                                                 {},
                                     train_event_log, n=N)

likelihoods_train = evaluator.sample_cases(plot_cases=False, multiprocessing=True)

  0%|          | 0/4960 [00:00<?, ?it/s]

100%|██████████| 4960/4960 [01:15<00:00, 65.38it/s] 


In [22]:
results = {
    "ll_train": np.mean([v.ln() for v in likelihoods_train[0].values()]),
    "ll_test": np.mean([v.ln() for v in likelihoods_test[0].values()]),
    "crsp_train": np.mean(get_pscores(likelihoods_train)),
    "crsp_test": np.mean(get_pscores(likelihoods_test))
}
results

{'ll_train': Decimal('-0.8048287057489850445137120550'),
 'll_test': Decimal('-0.8048287057489850445137120550'),
 'crsp_train': np.float64(11236.325480630368),
 'crsp_test': np.float64(11236.325480630368)}